### TO DO
* Kriging and IDW
* Sequential Code
* No Notion of Last K Days

In [5]:
from polire.interpolate import  Kriging, Idw
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [6]:
df = pd.read_csv("../data/beijinga_scaled.csv", index_col = 0)

In [7]:
timestamps = df['Time'].unique()
timestamps.sort()
stations = df['Station'].unique()

In [8]:
kf = KFold(n_splits=6, random_state=0, shuffle=True)

In [9]:
df.head()

,latitude,longitude,Time,Station,PM2.5
0,0.58292,0.17557,0.000000,1001.0,156.250000
0,0.58292,0.17557,0.002747,1001.0,24.833333
0,0.58292,0.17557,0.005495,1001.0,72.583333
0,0.58292,0.17557,0.008242,1001.0,30.043478
0,0.58292,0.17557,0.010989,1001.0,23.083333


In [10]:
for train_index, test_index in kf.split(stations):
    print(train_index, test_index)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 17 18 19 21 23 24 25 26 27
 28 29 32 33 34 35] [15 16 20 22 30 31]
[ 0  1  3  4  5  6  7  8  9 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26
 28 30 31 32 33 34] [ 2 10 11 27 29 35]
[ 0  1  2  3  4  6  7  9 10 11 12 14 15 16 17 18 19 20 21 22 23 24 25 26
 27 29 30 31 34 35] [ 5  8 13 28 32 33]
[ 0  2  3  4  5  6  8  9 10 11 13 15 16 18 19 20 21 22 23 24 25 27 28 29
 30 31 32 33 34 35] [ 1  7 12 14 17 26]
[ 0  1  2  3  5  7  8  9 10 11 12 13 14 15 16 17 19 20 21 22 26 27 28 29
 30 31 32 33 34 35] [ 4  6 18 23 24 25]
[ 1  2  4  5  6  7  8 10 11 12 13 14 15 16 17 18 20 22 23 24 25 26 27 28
 29 30 31 32 33 35] [ 0  3  9 19 21 34]


In [12]:
from sklearn.neighbors import KNeighborsRegressor

In [13]:
import gpflow
import tensorflow as tf

/home/deepak/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/deepak/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/deepak/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/deepak/anaconda3/envs/tf_gpu/lib/python3

In [22]:
rmse = {'K':{timestamp:{i:0 for i in range(6)} for timestamp in timestamps}, 'I':{timestamp:{i:0 for i in range(6)} for timestamp in timestamps},'KNN':{timestamp:{i:0 for i in range(6)} for timestamp in timestamps},
       'GP':{timestamp:{i:0 for i in range(6)} for timestamp in timestamps}}
mae = {'K':{timestamp:{i:0 for i in range(6)} for timestamp in timestamps}, 'I':{timestamp:{i:0 for i in range(6)} for timestamp in timestamps},'KNN':{timestamp:{i:0 for i in range(6)} for timestamp in timestamps},
      'GP':{timestamp:{i:0 for i in range(6)} for timestamp in timestamps}}


for timestamp in timestamps:
    
    print("TIMESTAMP", timestamp)
    i = -1 
    for train_index, test_index in kf.split(stations):
        i = i + 1
        train_stations = stations[train_index]
        test_stations = stations[test_index]
        try:
            train_df = pd.concat([df.groupby('Station').get_group(stn) for stn in train_stations])
            train_df = train_df.groupby('Time').get_group(timestamp)
            train_df.drop(columns = ['Station', 'Time'])
            test_df = pd.concat([df.groupby('Station').get_group(stn) for stn in test_stations])
            test_df = test_df.groupby('Time').get_group(timestamp)
            test_df.drop(columns = ['Station', 'Time'])

        except KeyError:
            print("FAIL")
            continue
        
        idw = Idw()
        krig = Kriging()
        knn = KNeighborsRegressor(n_neighbors=6)
        
        X_train = np.array( train_df[['latitude', 'longitude']] )
        y_train = np.array( train_df[['PM2.5']] )
        X_test = np.array( test_df[['longitude', 'latitude']] )
        y_test = np.array( test_df[['PM2.5']] )
        
        idw._fit(X_train, y_train)
        krig.fit(X_train, y_train)
        knn.fit(X_train, y_train)

        

        
        try:
            idw_res = idw._predict(X_test)
            krig_res = krig.predict(X_test)
            knn_res = knn.predict(X_test)
        
            idw_rms_error = np.sqrt(mean_squared_error(idw_res, y_test))
            krig_rms_error = np.sqrt(mean_squared_error(krig_res, y_test))
            idw_mae_error = mean_absolute_error(idw_res, y_test)
            krig_mae_error = mean_absolute_error(krig_res, y_test)
            knn_rms_error = np.sqrt(mean_squared_error(knn_res, y_test))
            knn_mae_error = mean_absolute_error(knn_res, y_test)
            
            

            tf.reset_default_graph()
            graph = tf.get_default_graph()
            gpflow.reset_default_session(graph=graph)


            xy_matern_1 = gpflow.kernels.Matern52(input_dim=2, ARD=True, active_dims=[0, 1])
            xy_matern_2 = gpflow.kernels.Matern52(input_dim=2, ARD=True, active_dims=[0, 1])



            kernel = xy_matern_1 + xy_matern_2


            model = gpflow.models.GPR(X_train, y_train, kern=kernel, mean_function=None)
            opt = gpflow.train.ScipyOptimizer()
            opt.minimize(model)

            mean, var = model.predict_y(X_test)
            
            gp_rms_error = np.sqrt(mean_squared_error(mean, y_test))
            gp_mae_error = mean_absolute_error(mean, y_test)

            
        except ValueError:
            continue
            
        
        rmse['K'][timestamp][i] = krig_rms_error
        rmse['I'][timestamp][i] = idw_rms_error
        rmse['KNN'][timestamp][i] = knn_rms_error
        rmse['GP'][timestamp][i] = gp_rms_error
        
        mae['K'][timestamp][i] = krig_mae_error
        mae['I'][timestamp][i] = idw_mae_error
        mae['KNN'][timestamp][i] = knn_rms_error
        mae['GP'][timestamp][i] = gp_mae_error
        
        

TIMESTAMP 0.0
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 123.450659
  Number of iterations: 48
  Number of functions evaluations: 86


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 123.450659
  Number of iterations: 48
  Number of functions evaluations: 86


Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().


Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 125.275448
  Number of iterations: 51
  Number of functions evaluations: 65


...

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 120.374093
  Number of iterations: 42
  Number of functions evaluations: 51


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 115.272845
  Number of iterations: 47
  Number of functions evaluations: 65


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 115.272845
  Number of iterations: 47
  Number of functions evaluations: 65


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 120.132198
  Number of iterations: 40
  Number of functions evaluations: 51


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 120.132198
  Number of iterations: 40
  Number of functions evaluations: 51


In [23]:
pd.DataFrame(rmse['GP']).mean(axis = 0).values.mean()

24.706916240745137

In [24]:
pd.DataFrame(rmse['K']).mean(axis = 0).values.mean()

24.78075242852069

In [25]:
pd.DataFrame(rmse['I']).mean(axis = 0).values.mean()

22.842467409712626

In [26]:
pd.DataFrame(rmse['KNN']).mean(axis = 0).values.mean()

22.914810690296544